In [6]:
stats = tfdv.generate_statistics_from_tfrecord(data_location="/data/tfrecord/alu-m2/Alu-M2-11201206.tfrecord")
tfdv.visualize_statistics(stats)

# create aoi-wzs-p3-dip-prewave-saiap.db

In [1]:
import os, glob, sqlite3

In [2]:
db_path = '/p3/aoi-wzs-p3-dip-prewave-saiap.db'

In [3]:
conn = sqlite3.connect(db_path)
c = conn.cursor()

In [7]:
# show all tables in a db
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[('labeled',)]

In [5]:
# drop selected table
c.execute('drop table labeled')
conn.commit()

In [6]:
# index cannot be column value
c.execute('''CREATE TABLE labeled
             (path text primary key, 
             filename text not null, 
             SN text, 
             board text, 
             location text not null, 
             component text, 
             degree text, 
             cap_value text, 
             voltage text, 
             index_count text, 
             extension text not null, 
             date text,  
             label text)''')
conn.commit()

In [8]:
# show all columns in a table
c.execute("PRAGMA table_info(labeled)")
c.fetchall()

[(0, 'path', 'text', 0, None, 1),
 (1, 'filename', 'text', 1, None, 0),
 (2, 'SN', 'text', 0, None, 0),
 (3, 'board', 'text', 0, None, 0),
 (4, 'location', 'text', 1, None, 0),
 (5, 'component', 'text', 0, None, 0),
 (6, 'degree', 'text', 0, None, 0),
 (7, 'cap_value', 'text', 0, None, 0),
 (8, 'voltage', 'text', 0, None, 0),
 (9, 'index_count', 'text', 0, None, 0),
 (10, 'extension', 'text', 1, None, 0),
 (11, 'date', 'text', 0, None, 0),
 (12, 'label', 'text', 0, None, 0)]

In [9]:
# keys for labeled
keys = ['path', 'filename', 'SN', 'board', 'location', 'component', 'degree', 'cap_value', 'voltage', 'index_count', 'extension', 'date', 'label']

In [62]:
def component_dict(component):
    component_lookup = {
        "Alu": "AluCap",
        "AluCap": "AluCap",
        "AluCapacitance": "AluCap",
        "AluCapacitor": "AluCap",
        "Elec": "ElecCap",
        "ElecCap": "ElecCap",
        "ElecCapacitance": "ElecCap",
        "ElecCapacitor": "ElecCap",
        "L": "L",
        "SATA": "SATA",
        "PCI": "PCI",
        "Leak": "unknown_from_path",
        "NG": "unknown_from_path",
    }
    return component_lookup.get(component, 'other_com')

def ng_class_dict(ng):
    ng_class_lookup = {
        "Leak": "NG-UnknownType",
        "OK":"OK",
        "Overkill":"OK",
        "NG": "NG-InversePolarity",
        "NG-InversePolarity":"NG-InversePolarity",
        "NG-OutsidePosition": "NG-OutsidePosition",
        "NG-TooHigh": "NG-OutsidePosition",
        "NG-UpsideDown": "NG-UpsideDown",
        "NG-MoreComp": "NG-MoreComp",
        "NG-MorethanAComponent": "NG-MoreComp",
        "NG-LackofComponent": "NG-NoneComp",
        "NG-NoneComponent": "NG-NoneComp",
        "NG-NoneComp": "NG-NoneComp",
    }
    return ng_class_lookup.get(ng, 'other_ng')

# insert metadata from Phase1-Cap

In [26]:
def parse_features_1(path):
    root, filename = os.path.split(path)
    name, ext = os.path.splitext(filename)
    r = root.split(os.path.sep)
    n = name.split('_')
    splitted_ext = ext.split('.')[-1]
    path = path.replace("p3", "data/aoi-wzs-p3-dip-prewave-saiap", 1)
    
    l = []
    if r[3][0]=='P':
        if r[-2]=='OK':
            l = [path, filename, '', '', n[0], component_dict(r[-1]), '-'+n[1], '', '', '', splitted_ext, '20190523', ng_class_dict(r[-2])]
        elif r[-3]=='NG':
            l = [path, filename, '', '', n[0], component_dict(r[-2]), '-'+n[1], '', '', '', splitted_ext, '20190523', ng_class_dict(r[-1])]
    elif r[3][0]=='D':
        if r[-2]=='OK':
            l = [path, filename, '', '', n[0], component_dict(r[-3]), '-'+r[-1].split('-')[0].split('e')[-1], '', '', '', splitted_ext, '', ng_class_dict(r[-1].split('-')[-1])]
        else:
            l = [path, filename, '', '', n[0], component_dict(r[-2]), '', '', '', '', splitted_ext, '', ng_class_dict(r[-1])]
    assert l != [], f'{path}'
#     for i, k in enumerate(keys):
#         print(k, l[i])
    
    return l

__./Phase1-Cap/P3_SAIAP_0523
COMPONENT_CLASS: AluCapacitance, ElecCapacitance__
`/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/P3_SAIAP_0523/DIP/OK/COMPONENT_CLASS/LOCATION_WrongDegree.bmp`
__NG_CLASS: NG-LackofComponent, NG-MorethanAComponent, NG-TooHigh, NG-UpsideDown__
`/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/P3_SAIAP_0523/DIP/NG/COMPONENT_CLASS/NG_CLASS/LOCATION_WrongDegree.bmp`


In [27]:
p3_saiap_ok = glob.glob('/p3/Phase1-Cap/P3_SAIAP_0523/DIP/OK/*/*.bmp')
for feature_list in map(parse_features_1, p3_saiap_ok):
    try:
        c.execute("INSERT INTO labeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()

Exception msg: UNIQUE constraint failed: labeled.path
/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/P3_SAIAP_0523/DIP/OK/ElecCapacitance/UTC72_270.bmp occured error while inserting


In [28]:
p3_saiap_0523_ng = glob.glob('/p3/Phase1-Cap/P3_SAIAP_0523/DIP/NG/*/*/*.bmp')
for feature_list in map(parse_features_1, p3_saiap_0523_ng):
    try:
        c.execute("INSERT INTO labeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()

Exception msg: UNIQUE constraint failed: labeled.path
/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/P3_SAIAP_0523/DIP/NG/ElecCapacitance/NG-InversePolarity/UTC72_270.bmp occured error while inserting


In [29]:
symbol = ('20190523', )
c.execute('SELECT * FROM labeled WHERE date=?', symbol)
c.fetchone()

('/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/P3_SAIAP_0523/DIP/OK/ElecCapacitance/UTC72_270.bmp',
 'UTC72_270.bmp',
 '',
 '',
 'UTC72',
 'ElecCap',
 '-270',
 '',
 '',
 '',
 'bmp',
 '20190523',
 'OK')

In [49]:
p3_saiap_0523_ok = '/p3/Phase1-Cap/P3_SAIAP_0523/DIP/OK/*/*.bmp'
parse_features_1(glob.glob(p3_saiap_0523_ok)[0])
p3_saiap_0523_ng = '/p3/Phase1-Cap/P3_SAIAP_0523/DIP/NG/*/*/*.bmp'
parse_features_1(glob.glob(p3_saiap_0523_ng)[0])

path /data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/P3_SAIAP_0523/DIP/OK/ElecCapacitance/UTC72_270.bmp
filename UTC72_270.bmp
SN 
board 
location UTC72
component ElecCap
degree -270
cap_value 
voltage 
index_count 
extension bmp
date 20190523
label OK
path /data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/P3_SAIAP_0523/DIP/NG/ElecCapacitance/NG-InversePolarity/UTC72_270.bmp
filename UTC72_270.bmp
SN 
board 
location UTC72
component ElecCap
degree -270
cap_value 
voltage 
index_count 
extension bmp
date 20190523
label NG-InversePolarity


__./Phase1-Cap/DIP_Data(2|3|4|5)___
__COMPONENT_CLASS: AluCapacitance, ElecCapacitance__
`/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/DIP_Data*/COMPONENT_CLASS/OK/Angle(0|90|180|270)-(OK|NG)/LOCATION.bmp`
__NG_CLASS: NG-LackofComponent, NG-MorethanAComponent, NG-TooHigh, NG-UpsideDown__
`/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/DIP_Data*/COMPONENT_CLASS/NG_CLASS/LOCATION.bmp`

In [30]:
DIP_OK = '/p3/Phase1-Cap/*/*/OK/*/*.bmp'
DIP_OK = [ok for ok in glob.glob(DIP_OK) if 'DIP' in ok.split(os.path.sep)[3]]
for feature_list in map(parse_features_1, DIP_OK):
    try:
        c.execute("INSERT INTO labeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
DIP_NG = '/p3/Phase1-Cap/*/*/*/*.bmp'
DIP_NG = [ng for ng in glob.glob(DIP_NG) if 'DIP' in ng.split(os.path.sep)[3] and 'NG' in ng.split(os.path.sep)[-2]]
for feature_list in map(parse_features_1, DIP_NG):
    try:
        c.execute("INSERT INTO labeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()

Exception msg: UNIQUE constraint failed: labeled.path
/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/DIP_Data5/ElecCapacitance/OK/Angle270-NG/TC5301_48 (6).bmp occured error while inserting
Exception msg: UNIQUE constraint failed: labeled.path
/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/DIP_Data5/ElecCapacitance/NG-LackofComponent/UC6_59 (2).bmp occured error while inserting


In [31]:
symbol = ('', )
c.execute('SELECT * FROM labeled WHERE date=?', symbol)
c.fetchone()

('/data/aoi-wzs-p3-dip-prewave-saiap/Phase1-Cap/DIP_Data5/ElecCapacitance/OK/Angle270-NG/TC5301_48 (6).bmp',
 'TC5301_48 (6).bmp',
 '',
 '',
 'TC5301',
 'ElecCap',
 '-270',
 '',
 '',
 '',
 'bmp',
 '',
 'NG-InversePolarity')

In [39]:
DIP_OK = '/p3/Phase1-Cap/*/*/OK/*/*.bmp'
DIP_OK = [ok for ok in glob.glob(DIP_OK) if 'DIP' in ok.split(os.path.sep)[3]]
parse_features_1(DIP_OK[0])
parse_features_1(DIP_OK[100])

DIP_NG = '/p3/Phase1-Cap/*/*/*/*.bmp'
DIP_NG = [ng for ng in glob.glob(DIP_NG) if 'DIP' in ng.split(os.path.sep)[3] and 'NG' in ng.split(os.path.sep)[-2]]
parse_features_1(DIP_NG[0])

path /p3/Phase1-Cap/DIP_Data5/ElecCapacitance/OK/Angle270-NG/TC5301_48 (6).bmp
filename TC5301_48 (6).bmp
SN 
board 
location TC5301
component ElecCap
degree -270
cap_value 
voltage 
index_count 
extension bmp
date 
label NG-InversePolarity
path /p3/Phase1-Cap/DIP_Data5/ElecCapacitance/OK/Angle270-OK/PT3201_43 (5).bmp
filename PT3201_43 (5).bmp
SN 
board 
location PT3201
component ElecCap
degree -270
cap_value 
voltage 
index_count 
extension bmp
date 
label OK
path /p3/Phase1-Cap/DIP_Data5/ElecCapacitance/NG-LackofComponent/UC6_59 (2).bmp
filename UC6_59 (2).bmp
SN 
board 
location UC6
component ElecCap
degree 
cap_value 
voltage 
index_count 
extension bmp
date 
label NG-NoneComp


In [26]:
conn.close()

# insert metadata from Phase2-Cap

In [35]:
def parse_features_2(path):
    root, filename = os.path.split(path)
    name, ext = os.path.splitext(filename)
    r = root.split(os.path.sep)
    n = name.split('_')
    splitted_ext = ext.split('.')[-1]
    path = path.replace("p3", "data/aoi-wzs-p3-dip-prewave-saiap", 1)
    
    l = []
    if r[2]=='Phase2-Cap' and int(r[3]) < 920 and int(r[3]) > 704:
        if r[5][-1]=='2':
            if ext == '.bmp':
                if len(n)==3:
                    if r[-2]=='OK':
                        l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), r[-1], '', '', '', splitted_ext, '2019'+r[3], ng_class_dict(r[-2])]
                    elif r[-2]=='NG':
                        l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), r[-1], '', '', '', splitted_ext, '2019'+r[3], ng_class_dict(r[-2])]
                if len(n)==2:
                    if r[-2]=='OK':
                        l = [path, filename, '', '', n[0], component_dict(r[4]), r[-1], '', '', '', splitted_ext, '2019'+r[3], ng_class_dict(r[-2])]
                    elif r[-2]=='NG':
                        l = [path, filename, '', '', n[0], component_dict(r[4]), r[-1], '', '', '', splitted_ext, '2019'+r[3], ng_class_dict(r[-2])]
            elif ext == '.png':
                if r[-2]=='OK':
                    l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), r[-1], n[-2], n[-1].split('(')[0], '', splitted_ext, '2019'+r[3], ng_class_dict(r[-2])]
                elif r[-2]=='NG':
                    l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), r[-1], n[-2], n[-1].split('(')[0], '', splitted_ext, '2019'+r[3], ng_class_dict(r[-2])]
        elif r[5][-1]=='1':
            if ext == '.bmp':
                if len(n)==3:
                    l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), '', '', '', '', splitted_ext, '2019'+r[3], ng_class_dict(r[-1])]
                elif len(n)==2:
                    l = [path, filename, '', '', n[0], component_dict(r[4]), '', '', '', '', splitted_ext, '2019'+r[3], ng_class_dict(r[-1])]
            elif ext == '.png':
                l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), '-'+n[2], n[-2], n[-1].split('(')[0], '', splitted_ext, '2019'+r[3], ng_class_dict(r[-1])]
    elif r[2]=='Phase2-Cap' and int(r[3]) > 919 and int(r[3]) < 1231:
        if r[-2]=='OK':
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[-3]), n[2], n[-2], n[-1].split('(')[0], '', splitted_ext, '2019'+r[3], ng_class_dict(r[-2])]
        else:
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[-2]), n[2], n[-2], n[-1].split('(')[0], '', splitted_ext, '2019'+r[3], ng_class_dict(r[-1])]
    elif r[2]=='Phase2-Cap' and int(r[3]) > 112 and int(r[3]) < 119:
        if r[-2]=='OK':
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(n[2]), n[3], n[-3], n[-2], n[-1], splitted_ext, '2020'+r[3], ng_class_dict(r[-2])]
        else:
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(n[2]), n[3], n[-3], n[-2], n[-1], splitted_ext, '2020'+r[3], ng_class_dict(r[-1])]
    assert l != [], f'{path}'
#     for i, k in enumerate(keys):
#         print(k, l[i])
    
    return l

### ./Phase2-Cap/[0705-0919]
##### directory tree showed below, btw some degree in the filename would be wrong.
```
# COMPONENT_CLASS: AluCapacitor, ElecCapacitor
# OK images in Model-2 
/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/DATE/COMPONENT_CLASS/Model-2/OK/DEGREE/BOARD_LOCATION_DEGREE_CAPACITY_VOLTAGE.png
# inverse polarity images in Model-2 
# mostly inverse polarity images will differ 180 from correct degree
/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/DATE/COMPONENT_CLASS/Model-2/NG/DEGREE/BOARD_LOCATION_DEGREE_CAPACITY_VOLTAGE.png
# NG_CLASS: NG-MoreComp, NG-NoneComp, NG-OutsidePosition, NG-UpsideDown
/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/DATE/COMPONENT_CLASS/Model-1/NG_CLASS/BOARD_LOCATION_DEGREE_CAPACITY_VOLTAGE.png
```


In [40]:
print("===model-2 ok===")
print("===model-2 inverse polarity===")
phase2_model2 = glob.glob('/p3/Phase2-Cap/*/*/*/*/*/*')
for feature_list in map(parse_features_2, phase2_model2):
    try:
        c.execute("INSERT INTO labeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()
print("===model-1 png===")
phase2_model1_png = glob.glob('/p3/Phase2-Cap/*/*/*/*/*.png')
for feature_list in map(parse_features_2, phase2_model1_png):
    try:
        c.execute("INSERT INTO labeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()
print("===model-1 bmp===")
phase2_model1_bmp = glob.glob('/p3/Phase2-Cap/*/*/*/*/*.bmp')
for feature_list in map(parse_features_2, phase2_model1_bmp):
    try:
        c.execute("INSERT INTO labeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()

===model-1 png===
===model-1 bmp===


In [41]:
print("===model-2 ok===")
phase2_m2_ok_png = '/p3/Phase2-Cap/*/*/*/OK/*/*.png'
parse_features_2(glob.glob(phase2_m2_ok_png)[0])
phase2_m2_ok_bmp = '/p3/Phase2-Cap/*/*/*/OK/*/*.bmp'
parse_features_2(glob.glob(phase2_m2_ok_bmp)[0])
parse_features_2(glob.glob(phase2_m2_ok_bmp)[2])
print("===model-2 inverse polarity===")
phase2_m2_ng_png = '/p3/Phase2-Cap/*/*/*/NG/*/*.png'
parse_features_2(glob.glob(phase2_m2_ng_png)[0])
phase2_m2_ng_bmp = '/p3/Phase2-Cap/*/*/*/NG/*/*.bmp'
parse_features_2(glob.glob(phase2_m2_ng_bmp)[0])
parse_features_2(glob.glob(phase2_m2_ng_bmp)[2])
print("===model-1===")
print("===0920-0118 ok===")
phase2_m1_png = '/p3/Phase2-Cap/*/*/*/*/*.png'
parse_features_2(glob.glob(phase2_m1_png)[0])
phase2_m1_bmp = '/p3/Phase2-Cap/*/*/*/*/*.bmp'
parse_features_2(glob.glob(phase2_m1_bmp)[0])
parse_features_2(glob.glob(phase2_m1_bmp)[1])

===model-2 ok===
path /p3/Phase2-Cap/0904/AluCapacitor/Model-2/OK/270/CN01W26NWS20099302YBA00_PT4701_90_NA_NA.png
filename CN01W26NWS20099302YBA00_PT4701_90_NA_NA.png
SN CN01W26NWS20099302YBA00_PT4701
board CN01W26NWS20099302YBA00
location PT4701
component AluCap
degree 270
cap_value NA
voltage NA
index_count 
extension png
date 20190904
label OK
path /p3/Phase2-Cap/0904/AluCapacitor/Model-2/OK/270/CN01W26NWS2009930250A00_PT4601_35.bmp
filename CN01W26NWS2009930250A00_PT4601_35.bmp
SN CN01W26NWS2009930250A00_PT4601
board CN01W26NWS2009930250A00
location PT4601
component AluCap
degree 270
cap_value 
voltage 
index_count 
extension bmp
date 20190904
label OK
path /p3/Phase2-Cap/0904/AluCapacitor/Model-2/OK/270/PT4901_1 (2).bmp
filename PT4901_1 (2).bmp
SN 
board 
location PT4901
component AluCap
degree 270
cap_value 
voltage 
index_count 
extension bmp
date 20190904
label OK
===model-2 inverse polarity===
path /p3/Phase2-Cap/0904/AluCapacitor/Model-2/NG/0/CN01W26NWS20099302G7A00_PT5101_0

### ./Phase2-Cap/[0920-1224]
##### directory tree showed below, btw all degree in the filename is true & corrected.
```
# COMPONENT_CLASS: AluCapacitor, ElecCapacitor
# OK images
/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/DATE/COMPONENT_CLASS/OK/DEGREE/BOARD_LOCATION_DEGREE_CAPACITY_VOLTAGE.png
# NG_CLASS: NG-MoreComp, (NG-NoneComp|NG-NoneComponent), NG-OutsidePosition, NG-UpsideDown, 
NG-InversePolarity
# mostly inverse polarity images will differ 180 from correct degree
/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/DATE/COMPONENT_CLASS/NG_CLASS/BOARD_LOCATION_DEGREE_CAPACITY_VOLTAGE.png
```

In [43]:
print("===0920-0118 ng===")
phase2_0920_0118_ng = glob.glob('/p3/Phase2-Cap/*/*/*/*.png')
for feature_list in map(parse_features_2, phase2_0920_0118_ng):
    try:
        c.execute("INSERT INTO labeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()

===0920-0118 ng===
Exception msg: UNIQUE constraint failed: labeled.path
/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/0926/AluCapacitor/NG-OutsidePosition/CN0YD74FWS20099Q00LEA00_PT4901_270_NA_NA.png occured error while inserting


In [44]:
conn.close()

In [42]:
phase2_2_ok = '/p3/Phase2-Cap/*/*/OK/*/*.png'
parse_features_2(glob.glob(phase2_2_ok)[0])
phase2_2_ng = '/p3/Phase2-Cap/0920/*/*/*.png'
parse_features_2(glob.glob(phase2_2_ng)[0])
phase2_2_ng_po = '/p3/Phase2-Cap/*/*/NG-InversePolarity/*.png'
parse_features_2(glob.glob(phase2_2_ng_po)[0])

path /p3/Phase2-Cap/0926/AluCapacitor/OK/270/CN0YD74FWS20099Q011RA00_PT4603_270_NA_NA.png
filename CN0YD74FWS20099Q011RA00_PT4603_270_NA_NA.png
SN CN0YD74FWS20099Q011RA00_PT4603
board CN0YD74FWS20099Q011RA00
location PT4603
component AluCap
degree 270
cap_value NA
voltage NA
index_count 
extension png
date 20190926
label OK
path /p3/Phase2-Cap/0920/AluCapacitor/NG-NoneComponent/CN01W26NWS20099I00IVA00_PT4506_0_NA_NA.png
filename CN01W26NWS20099I00IVA00_PT4506_0_NA_NA.png
SN CN01W26NWS20099I00IVA00_PT4506
board CN01W26NWS20099I00IVA00
location PT4506
component AluCap
degree 0
cap_value NA
voltage NA
index_count 
extension png
date 20190920
label NG-NoneComp
path /p3/Phase2-Cap/1005/AluCapacitor/NG-InversePolarity/CN0FPP7FWS2009A4008DA00_PT4506_0_NA_NA.png
filename CN0FPP7FWS2009A4008DA00_PT4506_0_NA_NA.png
SN CN0FPP7FWS2009A4008DA00_PT4506
board CN0FPP7FWS2009A4008DA00
location PT4506
component AluCap
degree 0
cap_value NA
voltage NA
index_count 
extension png
date 20191005
label NG-Inv

### ./Phase2-Cap/[0113-0118]
##### directory tree showed below, more completed filename updated
```
# COMPONENT_CLASS: AluCapacitor, ElecCapacitor
# OK images
/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/DATE/COMPONENT_CLASS/OK/DEGREE/BOARD_LOCATION_COMPONENT_DEGREE_CAPACITY_VOLTAGE_INDEX.png
# NG_CLASS: NG-MoreComp, (NG-NoneComp|NG-NoneComponent), NG-OutsidePosition, NG-UpsideDown, 
NG-InversePolarity
# mostly inverse polarity images will differ 180 from correct degree
/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/DATE/COMPONENT_CLASS/NG_CLASS/BOARD_LOCATION_COMPONENT_DEGREE_CAPACITY_VOLTAGE_INDEX.png
```

In [43]:
phase2_3_ok = '/p3/Phase2-Cap/0113/*/OK/*/*.png'
parse_features_2(glob.glob(phase2_3_ok)[0])
phase2_3_ng = '/p3/Phase2-Cap/0113/*/*/*.png'
parse_features_2(glob.glob(phase2_3_ng)[0])
phase2_3_ng_po = '/p3/Phase2-Cap/0115/*/NG-InversePolarity/*.png'
parse_features_2(glob.glob(phase2_3_ng_po)[0])

path /p3/Phase2-Cap/0113/ElecCapacitor/OK/270/CN0YD74FWS20001A00KXA00_TC3201_ElecCap_270_220_NA_0.png
filename CN0YD74FWS20001A00KXA00_TC3201_ElecCap_270_220_NA_0.png
SN CN0YD74FWS20001A00KXA00_TC3201
board CN0YD74FWS20001A00KXA00
location TC3201
component ElecCap
degree 270
cap_value 220
voltage NA
index_count 0
extension png
date 20200113
label OK
path /p3/Phase2-Cap/0113/AluCapacitor/NG-OutsidePosition/CN00010CWS20001B00PQA00_PT4504_AluCap_0_270_NA_0.png
filename CN00010CWS20001B00PQA00_PT4504_AluCap_0_270_NA_0.png
SN CN00010CWS20001B00PQA00_PT4504
board CN00010CWS20001B00PQA00
location PT4504
component AluCap
degree 0
cap_value 270
voltage NA
index_count 0
extension png
date 20200113
label NG-OutsidePosition
path /p3/Phase2-Cap/0115/ElecCapacitor/NG-InversePolarity/CN0YD74FWS20001D015OA00_TC3901_ElecCap_270_220_NA_0.png
filename CN0YD74FWS20001D015OA00_TC3901_ElecCap_270_220_NA_0.png
SN CN0YD74FWS20001D015OA00_TC3901
board CN0YD74FWS20001D015OA00
location TC3901
component ElecCap
d

# AIimg2020 to "table" unlabel

In [135]:
# show all tables in a db
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[('labeled',), ('unlabeled',)]

In [134]:
# index cannot be column value
c.execute('''CREATE TABLE unlabeled
             (path text primary key, 
             filename text not null, 
             SN text not null, 
             board text not null, 
             location text not null, 
             component text not null, 
             degree text, 
             cap_value text, 
             voltage text, 
             index_count text, 
             extension text not null, 
             eagle text,
             date text,  
             label text)''')
conn.commit()

In [110]:
c.execute("PRAGMA table_info(unlabeled)")
c.fetchall()

[(0, 'path', 'text', 0, None, 1),
 (1, 'filename', 'text', 1, None, 0),
 (2, 'SN', 'text', 1, None, 0),
 (3, 'board', 'text', 1, None, 0),
 (4, 'location', 'text', 1, None, 0),
 (5, 'component', 'text', 1, None, 0),
 (6, 'degree', 'text', 0, None, 0),
 (7, 'cap_value', 'text', 0, None, 0),
 (8, 'voltage', 'text', 0, None, 0),
 (9, 'index_count', 'text', 0, None, 0),
 (10, 'extension', 'text', 1, None, 0),
 (11, 'eagle', 'text', 0, None, 0),
 (12, 'date', 'text', 0, None, 0),
 (13, 'label', 'text', 0, None, 0)]

In [136]:
c.execute("select * from unlabeled")
c.fetchone()

In [137]:
# keys for unlabeled
keys = ['path', 'filename', 'SN', 'board', 'location', 'component', 'degree', 'cap_value', 'voltage', 'index_count', 'extension', 'eagle', 'date', 'label']

### ./AIimg2020/[1213-1228]
##### directory tree showed below
```
# COMPONENT_CLASS: AluCapacitor, ElecCapacitor
# OK/NG under EAGLE* is meaningless 
/data/aoi-wzs-p3-dip-prewave-saiap/AIimg2020/DATE/COMPONENT_CLASS/EAGLE\_.+\-2/(OK|NG)/BOARD_LOCATION_DEGREE_CAPACITY_VOLTAGE_INDEX.png
```

### ./AIimg2020/[0106-0118]
##### directory tree showed below, btw some degree in the filename would be wrong.
```
# COMPONENT_CLASS: AluCap, ElecCap, SATA, L, PCI
# OK/NG/TimeOut under EAGLE* is meaningless
/data/aoi-wzs-p3-dip-prewave-saiap/AIimg2020/DATE/COMPONENT_CLASS/EAGLE\_.+\-2/(OK|NG|TimeOut)/BOARD_LOCATION_COMPONENT_DEGREE_CAPACITY_VOLTAGE_INDEX.png
# EAGLE without -2 is the directory created by
the function of saving all cropped images no matter OK/NG judged by AOI
/data/aoi-wzs-p3-dip-prewave-saiap/AIimg2020/DATE/COMPONENT_CLASS/EAGLE\_.+[^\_][^2]/THE_SAME_COMPONENT_CLASS/DEGREE/BOARD_LOCATION_COMPONENT_DEGREE_CAPACITY_VOLTAGE_INDEX.png
```

In [84]:
# '/p3/AIimg2020/DATE/COMPONENT_CLASS/EAGLE\_.+[^\-][^2]/THE_SAME_COMPONENT_CLASS/DEGREE/BOARD_LOCATION_COMPONENT_DEGREE_CAPACITY_VOLTAGE_INDEX.png'
# Without -2
aiimg_6 = glob.glob('/p3/AIimg2020/*/*/*/*/*/*.png')
aiimg_6_no2 = [ai for ai in aiimg_6 if '-2' not in ai.split(os.path.sep)[5]]
aiimg_6_ng_0918 = [ai for ai in aiimg_6 if '-2' in ai.split(os.path.sep)[5]]
print(aiimg_6_no2[0])
print(aiimg_6_ng_0918[0])
print(aiimg_6_ng_0918[1])
# '/p3/AIimg2020/DATE/COMPONENT_CLASS/EAGLE\_.+\-2/(OK|NG|TimeOut)/BOARD_LOCATION_COMPONENT_DEGREE_CAPACITY_VOLTAGE_INDEX.png'
# With -2
aiimg_5 = glob.glob('/p3/AIimg2020/*/*/*/*/*.png')
print(aiimg_5[0])
aiimg_5_0106 = glob.glob('/p3/AIimg2020/0106/*/*/*/*.png')
print(aiimg_5_0106[0])

/p3/AIimg2020/0110/ElecCap/EAGLE_355_0DM01_0010002/ElecCap/270/CN01W26NWS20001803AWA00_TC3801_ElecCap_270_220_NA_0.png
/p3/AIimg2020/1226/NG/EAGLE_355_0DE01_0001-2/OK/0/CN00010CWS20099G00RTA00_PT4502_0_NA_NA.png
/p3/AIimg2020/0918/AluCapacitor/EAGLE_355_0DM01_0002_1-2/NG-NoneComponent/270/CN01W26NWS20099H02ZIA00_PT4901_270_NA_NA.png
/p3/AIimg2020/1226/AluCapacitor/EAGLE_355_0DE01_0001-2/OK/CN00010CWS2009CP00LUA00_PT5001_0_NA_NA.png
/p3/AIimg2020/0106/SATA/EAGLE_355_06K01_B001002-2/TimeOut/CN0R6JMPWS200014019IA00_SATA3_270_NA_NA_0.png


In [138]:
def parse_features_3(path):
    root, filename = os.path.split(path)
    name, ext = os.path.splitext(filename)
    r = root.split(os.path.sep)
    n = name.split('_')
    splitted_ext = ext.split('.')[-1]
    path = path.replace("p3", "data/aoi-wzs-p3-dip-prewave-saiap", 1)
    if int(r[3]) > 900:
        year = '2019'
    else: 
        year = '2020'
    
    l = []
    
    if r[5][-2] != '-':
        assert len(n)==7 or len(n)==6, f'{name}'
        if len(n)==7:
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(n[2]), n[3], n[4], n[5], n[6], splitted_ext, r[5], year+r[3], '']
        elif len(n)==6:
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), n[2], n[3], n[4], n[5], splitted_ext, r[5], year+r[3], '']
    else:
        if len(n)==5:
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), n[2], n[3], n[4], '', splitted_ext, r[5].split('-')[0], year+r[3], '']
        elif len(n)==6:
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(r[4]), n[2], n[3], n[4], n[5], splitted_ext, r[5].split('-')[0], year+r[3], '']
        elif len(n)==7:
            l = [path, filename, n[0]+'_'+n[1], n[0], n[1], component_dict(n[2]), n[3], n[4], n[5], n[6], splitted_ext, r[5].split('-')[0], year+r[3], '']
    assert l != [], f'{path}'
#     for i, k in enumerate(keys):
#         print(k, l[i])
    
    return l

In [99]:
aiimg_6 = glob.glob('/p3/AIimg2020/*/*/*/*/*/*.png')
aiimg_6_no2 = [ai for ai in aiimg_6 if '-2' not in ai.split(os.path.sep)[5]]
aiimg_6_ng_0918 = [ai for ai in aiimg_6 if '-2' in ai.split(os.path.sep)[5]]
parse_features_3(aiimg_6_no2[0])
parse_features_3(aiimg_6_ng_0918[0])
parse_features_3(aiimg_6_ng_0918[1])
aiimg_5 = glob.glob('/p3/AIimg2020/*/*/*/*/*.png')
parse_features_3(aiimg_5[0])
aiimg_5_0106 = glob.glob('/p3/AIimg2020/0106/*/*/*/*.png')
parse_features_3(aiimg_5_0106[0])

path /data/aoi-wzs-p3-dip-prewave-saiap/AIimg2020/0110/ElecCap/EAGLE_355_0DM01_0010002/ElecCap/270/CN01W26NWS20001803AWA00_TC3801_ElecCap_270_220_NA_0.png
filename CN01W26NWS20001803AWA00_TC3801_ElecCap_270_220_NA_0.png
SN CN01W26NWS20001803AWA00_TC3801
board CN01W26NWS20001803AWA00
location TC3801
component ElecCap
degree 270
cap_value 220
voltage NA
index_count 0
extension png
eagle EAGLE_355_0DM01_0010002
date 20200110
label 
path /data/aoi-wzs-p3-dip-prewave-saiap/AIimg2020/1226/NG/EAGLE_355_0DE01_0001-2/OK/0/CN00010CWS20099G00RTA00_PT4502_0_NA_NA.png
filename CN00010CWS20099G00RTA00_PT4502_0_NA_NA.png
SN CN00010CWS20099G00RTA00_PT4502
board CN00010CWS20099G00RTA00
location PT4502
component other_com
degree 0
cap_value NA
voltage NA
index_count 
extension png
eagle EAGLE_355_0DE01_0001
date 20191226
label 
path /data/aoi-wzs-p3-dip-prewave-saiap/AIimg2020/0918/AluCapacitor/EAGLE_355_0DM01_0002_1-2/NG-NoneComponent/270/CN01W26NWS20099H02ZIA00_PT4901_270_NA_NA.png
filename CN01W26NWS

In [139]:
aiimg_5 = glob.glob('/p3/AIimg2020/*/*/*/*/*.png')
for feature_list in map(parse_features_3, aiimg_5):
    try:
        c.execute("INSERT INTO unlabeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()

In [140]:
aiimg_6 = glob.glob('/p3/AIimg2020/*/*/*/*/*/*.png')
for feature_list in map(parse_features_3, aiimg_6):
    try:
        c.execute("INSERT INTO unlabeled VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", feature_list)
        c.fetchall()
    except Exception as e:
        print(f'Exception msg: {e}')
        print(f'{feature_list[0]} occured error while inserting')
        break
conn.commit()

In [142]:
conn.close()

# double check db

### update true com to where component="other_com"

In [6]:
c.execute('select distinct component from labeled')
c.fetchall()

[('ElecCap',), ('AluCap',), ('unknown_from_path',)]

In [7]:
c.execute('select count(*) from labeled where component="other_com"')
c.fetchone()

(0,)

In [234]:
c.execute('select * from labeled where component=="other_com"')
other_com_list = c.fetchall()

In [235]:
def update_other_com(ori_tuple):
    com_index = 5
    assert ori_tuple[com_index] == 'other_com', f'{ori_tuple[com_index]}'
    path = ori_tuple[0]
    splited_path = path.split(os.path.sep)
    true_com = ''
    if component_dict(splited_path[5]) != 'other_com':
        true_com = f'{component_dict(splited_path[5])}'
    if component_dict(splited_path[6]) != 'other_com':
        true_com = f'{component_dict(splited_path[6])}'
    assert true_com != '', f'{path}'
    sql_update_query = f"update labeled set component = '{true_com}' where path = '{path}'"
    return sql_update_query

In [236]:
update_count = 0
for sql in map(update_other_com, other_com_list):
#     print(sql)
    try:
        c.execute(sql)
        conn.commit()
        update_count+=1
    #     c.close()
    except sqlite3.Error as error:
        print('Failed to update sqlite table', error)
        print(update_count)
    except Exception as e:
        print(f'Exception msg: {e}')
        print(update_count)

In [246]:
c.execute('select * from labeled where component = "unknown_from_path"')
c.fetchone()

('/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/1114/Leak/InversePolarity/CN01W26NWS2009BF037EA00_PT4502_0_NA_NA.png',
 'CN01W26NWS2009BF037EA00_PT4502_0_NA_NA.png',
 'CN01W26NWS2009BF037EA00_PT4502',
 'CN01W26NWS2009BF037EA00',
 'PT4502',
 'unknown_from_path',
 '0',
 'NA',
 'NA',
 '',
 'png',
 '20191114',
 'other_ng')

### update true label to where component="other_ng"

In [33]:
c.execute('select distinct label from labeled')
c.fetchall()

[('OK',),
 ('NG-InversePolarity',),
 ('NG-NoneComp',),
 ('NG-OutsidePosition',),
 ('NG-UpsideDown',),
 ('NG-MoreComp',),
 ('NG-UnknownType',)]

In [34]:
c = conn.cursor()
c.execute('select count(*) from labeled where label="other_ng"')
c.fetchone()

(0,)

In [30]:
c.execute('select * from labeled where label=="other_ng"')
other_ng_list = c.fetchall()
other_ng_list[0]

('/data/aoi-wzs-p3-dip-prewave-saiap/Phase2-Cap/1005/AluCapacitor/Leak/CN0FPP7FWS2009A4008DA00_PT4506_0_NA_NA.png',
 'CN0FPP7FWS2009A4008DA00_PT4506_0_NA_NA.png',
 'CN0FPP7FWS2009A4008DA00_PT4506',
 'CN0FPP7FWS2009A4008DA00',
 'PT4506',
 'AluCap',
 '0',
 'NA',
 'NA',
 '',
 'png',
 '20191005',
 'other_ng')

In [31]:
def update_other_label(ori_tuple):
    label_index = -1
    assert ori_tuple[label_index] == 'other_ng', f'{ori_tuple[label_index]}'
    path = ori_tuple[0]
    splited_path = path.split(os.path.sep)
    true_label = ''
    if ng_class_dict(splited_path[6]) != 'other_ng':
        true_label = f'{ng_class_dict(splited_path[6])}'
    elif ng_class_dict(splited_path[5]) != 'other_ng':
        true_label = f'{ng_class_dict(splited_path[5])}'
    assert true_label != '', f'{path}'
    sql_update_query = f"update labeled set label = '{true_label}' where path = '{path}'"
    return sql_update_query

In [32]:
update_count = 0
for sql in map(update_other_label, other_ng_list):
#     print(sql)
    try:
        c.execute(sql)
        conn.commit()
        update_count+=1
    #     c.close()
    except sqlite3.Error as error:
        print('Failed to update sqlite table', error)
        print(update_count)
    except Exception as e:
        print(f'Exception msg: {e}')
        print(update_count)

### check component distinct of unlabeled

In [45]:
c.execute('select distinct component from unlabeled')
c.fetchall()

[('AluCap',),
 ('ElecCap',),
 ('unknown_from_path',),
 ('L',),
 ('SATA',),
 ('PCI',)]

In [46]:
c.execute('select distinct label from unlabeled')
c.fetchall()

[('',)]

In [49]:
c.execute('select * from unlabeled where component="unknown_from_path"')
# c.fetchone()
x = c.fetchall()
x

[('/data/aoi-wzs-p3-dip-prewave-saiap/AIimg2020/1226/NG/EAGLE_355_0DE01_0001-2/NG/CN00010CWS20099G00RTA00_PT4603_270_NA_NA.png',
  'CN00010CWS20099G00RTA00_PT4603_270_NA_NA.png',
  'CN00010CWS20099G00RTA00_PT4603',
  'CN00010CWS20099G00RTA00',
  'PT4603',
  'unknown_from_path',
  '270',
  'NA',
  'NA',
  '',
  'png',
  'EAGLE_355_0DE01_0001',
  '20191226',
  ''),
 ('/data/aoi-wzs-p3-dip-prewave-saiap/AIimg2020/1226/NG/EAGLE_355_0DE01_0001-2/OK/0/CN00010CWS20099G00RTA00_PT4502_0_NA_NA.png',
  'CN00010CWS20099G00RTA00_PT4502_0_NA_NA.png',
  'CN00010CWS20099G00RTA00_PT4502',
  'CN00010CWS20099G00RTA00',
  'PT4502',
  'unknown_from_path',
  '0',
  'NA',
  'NA',
  '',
  'png',
  'EAGLE_355_0DE01_0001',
  '20191226',
  '')]

In [43]:
def update_other_com(ori_tuple):
    com_index = 5
    assert ori_tuple[com_index] == 'other_com', f'{ori_tuple[com_index]}'
    path = ori_tuple[0]
    splited_path = path.split(os.path.sep)
    true_com = ''
    if component_dict(splited_path[5]) != 'other_com':
        true_com = f'{component_dict(splited_path[5])}'
    assert true_com != '', f'{path}'
    sql_update_query = f"update unlabeled set component = '{true_com}' where path = '{path}'"
    return sql_update_query

In [44]:
update_count = 0
for sql in map(update_other_com, other_com_list):
#     print(sql)
    try:
        c.execute(sql)
        conn.commit()
        update_count+=1
    #     c.close()
    except sqlite3.Error as error:
        print('Failed to update sqlite table', error)
        print(update_count)
    except Exception as e:
        print(f'Exception msg: {e}')
        print(update_count)

In [50]:
conn.close()